In [1]:
import pandas as pd
import numpy as np
from numpy import sort
import copy
import math
import cmath
import datetime

from sklearn.feature_selection import RFE,RFECV
from sklearn.feature_selection import SelectFromModel
import joblib


from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

import matplotlib.pyplot as plt

# Function

In [2]:

def calculate_LR(df1,df2):
    """
    df1:cv training set
    df2:cv test set
    """
    # log ((2+c)/(1+c)) + log ((2+c)/ (1+c)), {c==1}

    
    p = {}
    n = {}
    for index,row in df1.iterrows():
        if(pd.isna(row['ancestor'])):
            continue
        for i in row['ancestor'].split(','):
            if i not in p.keys():
                p[i]=1
                n[i]=1
            if(row['is_del']==1):
                p[i]+=1
            else:
                n[i]+=1
                
    l = copy.deepcopy(p)
    for i in l.keys():
        l[i]=math.log(p[i]/n[i]) 
    l
    
    def LR_add(x):
        sum=0
        if(pd.isna(x)):
            return sum
        for i in x.split(','):
            if i in l:
                sum = sum + l[i]
        return sum
    df1['LR'] = df1['ancestor'].apply(lambda x:LR_add(x))
    df2['LR'] = df2['ancestor'].apply(lambda x:LR_add(x))
    df1 = df1.drop(columns=['ancestor'])
    df2 = df2.drop(columns=['ancestor'])
    return df1,df2

In [3]:

def calculate_PA(df1,df2):
    """
    df1:cv training set
    df2:cv test set
    """
    # log ((2+c)/(1+c)) + log ((2+c)/ (1+c)), {c==1}


    p = {}
    n = {}
    for index,row in df1.iterrows():
        if(pd.isna(row['site'])):
            continue
        for i in row['site'].split(','):
            if i!='':
                if i not in p.keys():
                    p[i]=1
                    n[i]=1
                if(row['is_del']==1):
                    p[i]+=1
                else:
                    n[i]+=1
                
    s = copy.deepcopy(p)
    for i in s.keys():
        s[i]=math.log(p[i]/n[i]) 
    s
    

    def PA_add(x):
        sum=0
        if(pd.isna(x)):
            return sum
        for i in x.split(','):
            if i != '' and i in s:
                sum = sum + s[i]
        return sum
    df1['PA'] = df1['site'].apply(lambda x:PA_add(x))
    df2['PA'] = df2['site'].apply(lambda x:PA_add(x))
    df1 = df1.drop(columns=['site'])
    df2 = df2.drop(columns=['site'])
    return df1,df2

In [4]:

def tolerance_metrics(y_true, y_pre):

    #     y_true, y_pre = y_test1, p_test1
    label = pd.DataFrame({'true': y_true, 'pre': y_pre})


    unique_state = label.true.unique()
    targets = {}
    state_map = {1: 'p', 0: 'n', '0': 'p', '0': 'n'}
    tp = fp = tn = fn = 0
    for i, (t, p) in label.iterrows():
        if t == 0 and p == 0:
            tn += 1
        if t == 0 and p == 1:
            fp += 1
        if t == 1 and p == 1:
            tp += 1
        if t == 1 and p == 0:
            fn += 1

    allp = tp + fn
    alln = fp + tn


    N = tp + tn + fp + fn
    # ppv
    ppv = tp / (tp + fp)
    # npv
    npv = tn / (tn + fn)
    # sensitivity -> TPR
    sen = tp / (tp + fn)
    # spciticity -> TNR
    spe = tn / (tn + fp)
    # acc
    acc = (tp + tn) / N
    # MCC
    mcc = (tp*tn-fp*fn) /(((tp+fp) * (tp+fn) * (tn+fp) * (tn+fn))**0.5)
    # OPM
    opm = (ppv+npv)*(sen+spe)*(acc+(1+mcc)/2)/8
    columns = ['tp', 'tn', 'fp', 'fn', 'ppv', 'npv', 'tpr', 'tnr', 'acc', 'mcc', 'opm', 'N']
    res = pd.DataFrame(
        [
            [tp, tn, fp, fn, ppv, npv, sen, spe, acc, mcc, opm, N]
        ], 
        columns=columns,
    )
    

    return res.T

In [11]:
def sample(species):
    df1 = pd.read_csv('Dataset/All_species_train.csv')
    df2 = pd.read_csv('Dataset/Blind_{}.csv'.format(species))
    
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)

    for i in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("Blind_All_Species/{}/bootstrap_Combine_train_{}.csv".format(species,i),float_format='%.3f',index=None)

In [41]:

def no_reject_wGO(species,model,num):
    if(num!=-1):
        rfe = joblib.load('Feature_selected_1/lightgbm_feature_select_{}.rfe'.format(num))
    
    Blind_train = pd.read_csv('Dataset/All_species_train.csv')
    Blind_test = pd.read_csv('Dataset/Blind_{}.csv'.format(species))

    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)

    y_test = df2.is_del.values
    X_test = df2.iloc[:,df2.columns != "nutation"].iloc[:,5:]
    y_train = df1.is_del.values
    X_train = df1.iloc[:,df1.columns != "nutation"].iloc[:,5:]
    
    if(num!=-1):
        model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
        y_pred = model.predict(pd.DataFrame(rfe.transform(X_test)))
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    performance = tolerance_metrics(y_test, y_pred)

    performance.to_csv('out/Blind_All/{}_re_{}Fs_all.csv'.format(species,num))
    return performance

In [65]:

def reject_wGO(species,model,num):
    if(num!=-1):
        rfe = joblib.load('Feature_selected_1/lightgbm_feature_select_{}.rfe'.format(num))
    
    Blind_train = pd.read_csv('Dataset/All_species_train.csv')
    Blind_test = pd.read_csv('Dataset/Blind_{}.csv'.format(species))

    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)

    y_test = df2.is_del.values
    X_test = df2.iloc[:,df2.columns != "nutation"].iloc[:,5:]
    for i in range(200):
        data1 = pd.read_csv("Blind_All_Species_2/{}/bootstrap_Combine_train_{}.csv".format(species,i))
        y_train = data1.is_del.values
        X_train = data1.iloc[:,data1.columns != "nutation"].iloc[:,5:]
        if(num!=-1):
            model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
            p_test = model.predict_proba(pd.DataFrame(rfe.transform(X_test)))
        else:
            model.fit(X_train, y_train)
            p_test = model.predict_proba(X_test)
        df = pd.DataFrame(p_test[:,-1],df2.index)
        df.to_csv("Blind_All_Species_2/{}/Blind{}Fs/bootstrap_re_lgbm/bootstrap_Combine_{}_re.csv".format(species,num,i))

    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)

    data1 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs/bootstrap_re_lgbm/bootstrap_Combine_0_re.csv".format(species,num))
    for i in range(1,200):
        data2 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs/bootstrap_re_lgbm/bootstrap_Combine_{}_re.csv".format(species,num,i))
        data1 = pd.concat([data1,data2], ignore_index=True)

    d_mean = data1.groupby('Unnamed: 0')['0'].mean()
    d_std = data1.groupby('Unnamed: 0')['0'].std()
    data1 = pd.merge(pd.DataFrame(d_mean),pd.DataFrame(d_std),on='Unnamed: 0',how='outer')
    data1


    k = 20 ** 0.5
    def getA(row):
        return row['0_x']-k*row['0_y']

    def getB(row):
        return row['0_x']+k*row['0_y']
    data1["A"]=data1.apply(lambda row:getA(row),axis=1)
    data1["B"]=data1.apply(lambda row:getB(row),axis=1)
    data1

    def getT(row):
        if (row['A']<0.5) and (row['B']>0.5):
            return 1
        return 0
    data1["T"]=data1.apply(lambda row:getT(row),axis=1)
    data1

    data2 = pd.merge(data1[['T']],df2,left_index=True,right_index=True,how='outer')
    data2

    data3 = data2[data2['T']==0]
    data3

    del data3['T']
    data3

    data3.to_csv("Blind_All_Species_2/{}/Blind{}Fs/Combine_train_lgbm.csv".format(species,num),index=False,float_format='%.3f')

    y_train = df1.is_del.values
    X_train = df1.iloc[:,df1.columns != "nutation"].iloc[:,5:]
    data3 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs/Combine_train_lgbm.csv".format(species,num))
    y_test = data3.is_del.values
    X_test = data3.iloc[:,data3.columns != "nutation"].iloc[:,5:]
    
    if(num!=-1):
        model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
        y_pred=model.predict(pd.DataFrame(rfe.transform(X_test)))
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
            
    performance = tolerance_metrics(y_test, y_pred)

    performance.to_csv('out/Blind_All/{}_re_{}Fs_wGO.csv'.format(species,num))
    return performance

In [43]:

def no_reject_woGO(species,model,num):
    if(num!=-1):
        rfe = joblib.load('Feature_selected_2/lightgbm_feature_select_{}_N.rfe'.format(num))
    
    Blind_train = pd.read_csv('Dataset/All_species_train.csv')
    Blind_train
    Blind_test = pd.read_csv('Dataset/Blind_{}.csv'.format(species))
    Blind_test

    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)

    y_test = df2.is_del.values
    df2 = df2.iloc[:,df2.columns != 'nutation'].iloc[:,5:].iloc[:,:-1]
    X_test = df2.iloc[:,df2.columns != 'ancestor']
    y_train = df1.is_del.values
    df1 = df1.iloc[:,df1.columns != 'nutation'].iloc[:,5:].iloc[:,:-1]
    X_train = df1.iloc[:,df1.columns != 'ancestor']
    
    if(num!=-1):
        model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
        y_pred = model.predict(pd.DataFrame(rfe.transform(X_test)))
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    performance = tolerance_metrics(y_test, y_pred)
    performance.to_csv('out/Blind_All/{}_re_{}Fs_N_all.csv'.format(species,num))
    return performance

In [66]:

def reject_woGO(species,model,num):
    if(num!=-1):
        rfe = joblib.load('Feature_selected_2/lightgbm_feature_select_{}_N.rfe'.format(num))
    
    Blind_train = pd.read_csv('Dataset/All_species_train.csv')
    Blind_train
    Blind_test = pd.read_csv('Dataset/Blind_{}.csv'.format(species))
    Blind_test

    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)

    y_test = df2.is_del.values
    df = df2.iloc[:,df2.columns != 'nutation'].iloc[:,5:].iloc[:,:-1]
    X_test = df.iloc[:,df.columns != 'ancestor']
    for i in range(200):
        data1 = pd.read_csv("Blind_All_Species_2/{}/bootstrap_Combine_train_{}.csv".format(species,i))
        y_train = data1.is_del.values
        X_train = data1.iloc[:,data1.columns != 'nutation'].iloc[:,5:].iloc[:,:-2]
        
        if(num!=-1):
            model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
            p_test = model.predict_proba(pd.DataFrame(rfe.transform(X_test)))
        else:
            model.fit(X_train, y_train)
            p_test = model.predict_proba(X_test)
            
        df = pd.DataFrame(p_test[:,-1],df2.index)
        df.to_csv("Blind_All_Species_2/{}/Blind{}Fs_N/bootstrap_re_lgbm/bootstrap_Combine_{}_re.csv".format(species,num,i))
        
        
    df1 = copy.deepcopy(Blind_train)
    df2 = copy.deepcopy(Blind_test)

    data1 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs_N/bootstrap_re_lgbm/bootstrap_Combine_0_re.csv".format(species,num))
    for i in range(1,200):
        data2 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs_N/bootstrap_re_lgbm/bootstrap_Combine_{}_re.csv".format(species,num,i))
        data1 = pd.concat([data1,data2], ignore_index=True)

    d_mean = data1.groupby('Unnamed: 0')['0'].mean()
    d_std = data1.groupby('Unnamed: 0')['0'].std()

    data1 = pd.merge(d_mean,d_std,on='Unnamed: 0',how='outer')
    data1


    k = 20 ** 0.5
    def getA(row):
        return row['0_x']-k*row['0_y']

    def getB(row):
        return row['0_x']+k*row['0_y']
    data1["A"]=data1.apply(lambda row:getA(row),axis=1)
    data1["B"]=data1.apply(lambda row:getB(row),axis=1)
    data1


    def getT(row):
        if (row['A']<0.5) and (row['B']>0.5):
            return 1
        return 0
    data1["T"]=data1.apply(lambda row:getT(row),axis=1)
    data1

    data2 = pd.merge(data1[['T']],df2,left_index=True,right_index=True,how='outer')
    data2

    data3 = data2[data2['T']==0]
    data3


    del data3['T']
    data3

    data3.to_csv("Blind_All_Species_2/{}/Blind{}Fs_N/Combine_train_lgbm.csv".format(species,num),index=False,float_format='%.3f')


    y_train = df1.is_del.values
    df1 = df1.iloc[:,df1.columns != 'nutation'].iloc[:,5:].iloc[:,:-1]
    X_train = df1.iloc[:,df1.columns != 'ancestor']

    data3 = pd.read_csv("Blind_All_Species_2/{}/Blind{}Fs_N/Combine_train_lgbm.csv".format(species,num))
    y_test = data3.is_del.values
    data3 = data3.iloc[:,data3.columns != 'nutation'].iloc[:,5:].iloc[:,:-1]
    X_test = data3.iloc[:,data3.columns != 'ancestor']
    
    if(num!=-1):
        model.fit(pd.DataFrame(rfe.transform(X_train)), y_train)
        y_pred=model.predict(pd.DataFrame(rfe.transform(X_test)))
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    performance = tolerance_metrics(y_test, y_pred)

    performance.to_csv('out/Blind_All/{}_re_{}Fs_N.csv'.format(species,num))
    return performance

In [99]:
def train_1(species,model,num):
    performance_1 = no_reject_wGO(species,model,num)
    performance_1.columns = ['wGO_N']
    performance_2 = reject_wGO(species,model,num)
    performance_2.columns = ['wGO']
    performance_3 = no_reject_woGO(species,model,num)
    performance_3.columns = ['woGO_N']
    performance_4 = reject_woGO(species,model,num)
    performance_4.columns = ['woGO']
    performance = pd.concat([performance_1, performance_2, performance_3, performance_4],axis=1)
    performance.to_csv('out/Blind_All/{}_{}Fs.csv'.format(species,num))
    return performance

# Sample

In [12]:
sample('All_species')
sample('Human')
sample('Plant')
sample('Animal')

# Trian

In [100]:
def train_2(species,model):
#     num = 10
#     train_1(species,model,num)
    num = 20
    train_1(species,model,num)
#     num = 50
#     train_1(species,model,num)
#     num = 100
#     train_1(species,model,num)
#     num = -1
#     train_1(species,model,num)

#### All_species

In [104]:
species = 'All_species'
model = lgb.LGBMClassifier()
train_2(species,model)

#### Human

In [103]:
species = 'Human'
model = lgb.LGBMClassifier()
train_2(species,model)

#### Plant

In [102]:
species = 'Plant'
model = lgb.LGBMClassifier()
train_2(species,model)

#### Animal

In [101]:
species = 'Animal'
model = lgb.LGBMClassifier()
train_2(species,model)